## Decision Tree for Spam 
### (make sure to have run featurize_spam.py)

In [1]:
import numpy as np
np.random.seed(1)

import scipy.io
from sklearn.model_selection import train_test_split

from decisiontree import *

In [ ]:
spam_path = "datasets/spam-dataset/spam_data.mat"
data = scipy.io.loadmat(spam_path)
x = data["training_data"]
y = np.squeeze(data["training_labels"])
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)
x_test = data["test_data"]
x.shape,y.shape,x_test.shape

((5172, 25), (5172,), (5857, 25))

In [ ]:
%%time
dt = DecisionTree()
dt.fit(x_train, y_train)

In [ ]:
train_preds = dt.predict(x_train,"train")
print("training accuracy:",np.mean(train_preds==y_train))
val_preds = dt.predict(x_val,"val")
print("validation accuracy:",np.mean(val_preds==y_val))
dt.predict(x_test,"test")